In [11]:
#This section is for imports and globals/configs

#Credits to https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

import matplotlib
#matplotlib.use('TkAgg')
#import matplotlib.pyplot as plt
#get_ipython().run_line_magic('config', "InlineBackend.figure_format = 'png'")
#get_ipython().run_line_magic('matplotlib', 'inline')

from music21 import converter, instrument, note, chord
import pandas as pd
import numpy as np
import glob

In [13]:
#This section is for reading in the midi file into python

notes = []

#read in midi files from path
midi_path = "midi_songs" #need to add a directory of midi music here :D
for file in glob.glob(midi_path+"/*.midi"):
    midi = converter.parse(file)
    notes_to_parse = None
    
    parts = instrument.partitionByInstrument(midi)
    
    #check if file has instrument parts
    if parts:
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes
        
    for element in notes_to_parse:
        #check to see if this is a note or chord
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
            

In [ ]:
#This section is for translating the input from strings to numbers (easier for the RNN to parse)

